# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'09-30-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'09-30-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-01 04:23:42,33.93911,67.709953,39268,1458,32789,5021.0,Afghanistan,100.872524,3.712947
1,NaN,NaN,NaN,Albania,2020-10-01 04:23:42,41.15330,20.168300,13649,387,7847,5415.0,Albania,474.285913,2.835373
2,NaN,NaN,NaN,Algeria,2020-10-01 04:23:42,28.03390,1.659600,51530,1736,36174,13620.0,Algeria,117.511458,3.368911
3,NaN,NaN,NaN,Andorra,2020-10-01 04:23:42,42.50630,1.521800,2050,53,1432,565.0,Andorra,2653.206497,2.585366
4,NaN,NaN,NaN,Angola,2020-10-01 04:23:42,-11.20270,17.873900,4972,183,1941,2848.0,Angola,15.127973,3.680611


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39074,39096,39145,39170,39186,39192,39227,39233,39254,39268
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,12535,12666,12787,12921,13045,13153,13259,13391,13518,13649
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,50023,50214,50400,50579,50754,50914,51067,51213,51368,51530


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1444,1445,1446,1451,1451,1453,1453,1455,1458,1458
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,364,367,370,370,373,375,377,380,384,387
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1679,1689,1698,1703,1707,1711,1714,1719,1726,1736


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32576,32576,32610,32619,32619,32635,32642,32642,32746,32789
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6995,7042,7139,7239,7309,7397,7397,7629,7732,7847
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,35180,35307,35428,35544,35654,35756,35860,35962,36063,36174


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1691,1714,1715,1738,1757,1764,1773,1785,1787,1791
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5087,5124,5141,5165,5456,5477,5526,5588,5606,5640
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,838,848,851,857,873,882,885,886,886,896


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,24,25,25,25,25,25,25,25,27,27
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,49,49,49,50,50,50,50,50,50,52
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
632,1001.0,Autauga,Alabama,US,2020-10-01 04:23:42,32.539527,-86.644082,1791,27,0,1764.0,"Autauga, Alabama, US",3205.713365,1.507538
669,1075.0,Lamar,Alabama,US,2020-10-01 04:23:42,33.779950,-88.096680,382,2,0,380.0,"Lamar, Alabama, US",2767.113365,0.523560
670,1077.0,Lauderdale,Alabama,US,2020-10-01 04:23:42,34.901719,-87.656247,1759,34,0,1725.0,"Lauderdale, Alabama, US",1896.925449,1.932916


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,7233042,206932,2840688
India,6225763,97497,5187825
Brazil,4810935,143952,4274774


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,7233042,206932,2840688,4185422,2020-10-01 04:23:42,37.923092,-91.345036
India,6225763,97497,5187825,940441,2020-10-01 04:23:42,23.088275,81.806127
Brazil,4810935,143952,4274774,392209,2020-10-01 04:23:42,-12.669522,-48.480493
Russia,1170799,20630,954788,195381,2020-10-01 04:23:42,54.546312,62.120860
Colombia,829679,25998,743653,60028,2020-10-01 04:23:42,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,819115,15885,0
Texas,773019,16016,0
Florida,706516,14317,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,819115,15885,0,803230,2020-10-01 04:23:42,37.843962,-120.728594
Texas,773019,16016,0,757003,2020-10-01 04:23:42,31.660643,-98.653069
Florida,706516,14317,0,692199,2020-10-01 04:23:42,28.940755,-82.700744
New York,458649,33153,0,425496,2020-10-01 04:23:42,42.544151,-75.474183
Georgia,318026,7021,0,311005,2020-10-01 04:23:42,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,270299,6576,0
Florida,Miami-Dade,170400,3280,0
Illinois,Cook,145462,5225,0
Texas,Harris,142681,2599,0
Arizona,Maricopa,141856,3392,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,270299,6576,0,263723,2020-10-01 04:23:42,34.308284,-118.228241,6037.0
Florida,Miami-Dade,170400,3280,0,167120,2020-10-01 04:23:42,25.611236,-80.551706,12086.0
Illinois,Cook,145462,5225,0,140237,2020-10-01 04:23:42,41.841448,-87.816588,17031.0
Texas,Harris,142681,2599,0,140082,2020-10-01 04:23:42,29.858649,-95.393395,48201.0
Arizona,Maricopa,141856,3392,0,138464,2020-10-01 04:23:42,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,39233,13391,51213,1966,4797,101,723132,49574,27055,43432,...,441573,2010,55776,73528,1077,39121,10,2031,14660,7816
2020-09-29,39254,13518,51368,1966,4905,101,736609,49901,27078,44041,...,448729,2033,56354,74363,1094,39541,10,2031,14715,7837
2020-09-30,39268,13649,51530,2050,4972,101,751001,50359,27096,44813,...,455846,2046,56717,75122,1094,39899,10,2034,14759,7838


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,1455,380,1719,53,176,3,16113,957,882,790,...,42090,47,460,614,35,299,1,587,332,228
2020-09-29,1458,384,1726,53,179,3,16519,958,886,796,...,42162,48,466,621,35,306,1,587,332,228
2020-09-30,1458,387,1736,53,183,3,16937,959,888,799,...,42233,48,470,628,35,311,1,587,332,228


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,32642,7629,35962,1265,1813,92,576715,43665,24679,34052,...,2352,1755,52324,63346,999,30220,8,1266,13821,6112
2020-09-29,32746,7732,36063,1265,1833,92,585857,43849,24750,34916,...,2368,1771,52970,64200,1007,31047,8,1275,13937,6122
2020-09-30,32789,7847,36174,1432,1941,92,594645,44001,24782,35644,...,2373,1791,53366,65225,1010,31743,8,1286,13959,6303


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1691,1714,1715,1738,1757,1764,1773,1785,1787,1791
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,353,355,357,363,366,367,370,377,382,382
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1653,1662,1673,1682,1698,1713,1723,1740,1751,1759
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,592,603,605,609,616,620,621,628,634,636
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,5751,5765,5792,5829,5887,5897,5927,5945,5971,6017


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-28,152983,7597,0,217510,82049,812711,69469,57147,20389,49,...,193732,761198,71442,1745,1318,146444,86638,15516,117588,5754
2020-09-29,153554,7722,0,218184,82755,815654,70004,57329,20531,49,...,194611,767264,72136,1749,1318,147345,87042,15693,119955,5821
2020-09-30,154701,7824,0,218507,83697,819115,70515,57550,20613,49,...,196139,773019,73042,1752,1318,148092,87522,15850,122274,5948


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-28        1785    5588     886  657   1618     607    914    3548   
2020-09-29        1787    5606     886  658   1621     607    917    3556   
2020-09-30        1791    5640     896  664   1629     610    917    3569   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-28         1172      614  ...     238     30      292      100   
2020-09-29         1175      617  ...     240     32      295      108   
2020-09-30         1179      620  ...     245     33      296      116   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-28            334   560   354          0      115     36  
2020-09-29            336   560   354          0      116     37  
2020-09-30            339   577   357          0      116     39  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-28,2501,56,0,5624,1329,15633,2044,4503,634,0,...,2389,15807,453,58,20,3170,2100,342,1283,50
2020-09-29,2517,56,0,5632,1350,15782,2046,4505,635,0,...,2420,15902,457,58,20,3185,2124,350,1300,50
2020-09-30,2540,56,0,5650,1369,15885,2051,4508,636,0,...,2454,16016,459,58,20,3205,2126,355,1327,50


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-09-28          25      50       7   10     15      14     39      44   
2020-09-29          27      50       7   10     15      14     40      44   
2020-09-30          27      52       7   11     15      15     40      44   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-09-28           42       13  ...       2      1        4        1   
2020-09-29           42       13  ...       2      1        4        1   
2020-09-30           42       13  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-28              2     1     2          0        6      0  
2020-09-29              2     1     2          0        6      0  
2020-09-30              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,0.000153,0.009956,0.002859,0.070806,0.016744,0.0,0.016599,0.003522,0.000407,0.012968,...,0.009270,0.000996,0.008243,0.011514,0.002793,0.010800,0.0,0.000493,0.001298,0.000512
2020-09-29,0.000535,0.009484,0.003027,0.000000,0.022514,0.0,0.018637,0.006596,0.000850,0.014022,...,0.016206,0.011443,0.010363,0.011356,0.015785,0.010736,0.0,0.000000,0.003752,0.002687
2020-09-30,0.000357,0.009691,0.003154,0.042726,0.013660,0.0,0.019538,0.009178,0.000665,0.017529,...,0.015860,0.006394,0.006441,0.010207,0.000000,0.009054,0.0,0.001477,0.002990,0.000128


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,0.001376,0.007958,0.002917,0.0,0.011494,0.0,0.023113,0.006309,0.008000,0.003812,...,0.000309,0.000000,0.004367,0.013201,0.0,0.027491,0.0,0.0,0.0,0.004405
2020-09-29,0.002062,0.010526,0.004072,0.0,0.017045,0.0,0.025197,0.001045,0.004535,0.007595,...,0.001711,0.021277,0.013043,0.011401,0.0,0.023411,0.0,0.0,0.0,0.000000
2020-09-30,0.000000,0.007812,0.005794,0.0,0.022346,0.0,0.025304,0.001044,0.002257,0.003769,...,0.001684,0.000000,0.008584,0.011272,0.0,0.016340,0.0,0.0,0.0,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,0.000000,0.031364,0.002844,0.001584,0.062097,0.0,0.019048,0.001192,0.001949,0.013784,...,0.005558,0.015625,0.009551,0.014721,0.000000,0.039631,0.0,0.000000,0.002684,0.000983
2020-09-29,0.003186,0.013501,0.002809,0.000000,0.011031,0.0,0.015852,0.004214,0.002877,0.025373,...,0.006803,0.009117,0.012346,0.013482,0.008008,0.027366,0.0,0.007109,0.008393,0.001636
2020-09-30,0.001313,0.014873,0.003078,0.132016,0.058920,0.0,0.015000,0.003466,0.001293,0.020850,...,0.002111,0.011293,0.007476,0.015966,0.002979,0.022418,0.0,0.008627,0.001579,0.029566


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-28,0.004346,0.015506,NaN,0.001257,0.009933,0.003483,0.005952,0.009896,0.006367,0.0,...,0.003819,0.003218,0.011711,0.001722,0.000759,0.003034,0.004277,0.010748,0.014897,0.021481
2020-09-29,0.003732,0.016454,NaN,0.003099,0.008605,0.003621,0.007701,0.003185,0.006965,0.0,...,0.004537,0.007969,0.009714,0.002292,0.000000,0.006153,0.004663,0.011408,0.020130,0.011644
2020-09-30,0.007470,0.013209,NaN,0.001480,0.011383,0.004243,0.007300,0.003855,0.003994,0.0,...,0.007852,0.007501,0.012560,0.001715,0.000000,0.005070,0.005515,0.010004,0.019332,0.021818


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-28      0.006768  0.011220  0.001130  0.001524  0.000618  0.001650   
2020-09-29      0.001120  0.003221  0.000000  0.001522  0.001854  0.000000   
2020-09-30      0.002238  0.006065  0.011287  0.009119  0.004935  0.004942   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-28      0.001095  0.003962  0.003425  0.003268  ...  0.008475   
2020-09-29      0.003282  0.002255  0.002560  0.004886  ...  0.008403   
2020-09-30      0.000000  0.003656  0.003404  0.004862  ...  0.020833   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-28      0.071429  0.010381  0.030928  -0.002985  0.010830  0.005682   
2020-09-29      0.066667  0.010274  0.080000   0.005988  0.000000  0.000000   
2020-09-30      0.031250  0.003390  0.074074   0.008929  0.030357  0.008475   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-28            NaN  0.000000  0.000000  
2020-09-29            NaN  0.008696  0.027778  
2020-09-30            NaN  0.000000  0.054054  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-28,0.000000,0.0,NaN,0.000178,0.016055,0.001730,0.001470,0.000444,0.001580,NaN,...,0.005048,0.003938,0.000000,0.0,0.052632,0.004118,0.000000,0.008850,0.001561,0.0
2020-09-29,0.006397,0.0,NaN,0.001422,0.015801,0.009531,0.000978,0.000444,0.001577,NaN,...,0.012976,0.006010,0.008830,0.0,0.000000,0.004732,0.011429,0.023392,0.013250,0.0
2020-09-30,0.009138,0.0,NaN,0.003196,0.014074,0.006526,0.002444,0.000666,0.001575,NaN,...,0.014050,0.007169,0.004376,0.0,0.000000,0.006279,0.000942,0.014286,0.020769,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga Baldwin Barbour Bibb Blount   Bullock    Butler   
2020-09-28        0.00    0.00     0.0  0.0    0.0  0.000000  0.000000   
2020-09-29        0.08    0.00     0.0  0.0    0.0  0.000000  0.025641   
2020-09-30        0.00    0.04     0.0  0.1    0.0  0.071429  0.000000   

Province_State                            ... Wyoming                  \
Admin2         Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-09-28         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2020-09-29         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2020-09-30         0.0      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-09-28          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-29          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-09-30          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,0.000399,0.009282,0.003022,0.039357,0.016373,0.008470,0.015873,0.005406,0.000432,0.014787,...,0.011659,0.004359,0.008714,0.011176,0.002582,0.010776,1.753024e-31,0.000322,0.002394,0.001258
2020-09-29,0.000467,0.009383,0.003024,0.019678,0.019444,0.004235,0.017255,0.006001,0.000641,0.014404,...,0.013932,0.007901,0.009539,0.011266,0.009183,0.010756,8.765121e-32,0.000161,0.003073,0.001973
2020-09-30,0.000412,0.009537,0.003089,0.031202,0.016552,0.002118,0.018396,0.007590,0.000653,0.015967,...,0.014896,0.007148,0.007990,0.010736,0.004592,0.009905,4.382561e-32,0.000819,0.003031,0.001050


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,0.001000,0.006752,0.002591,6.832142e-17,0.016235,5.387291e-51,0.021287,0.004426,0.005624,0.002722,...,0.000464,0.000345,0.007155,0.012985,5.926234e-10,0.023740,0.0,0.000111,0.000069,0.002311
2020-09-29,0.001531,0.008639,0.003332,3.416071e-17,0.016640,2.693646e-51,0.023242,0.002736,0.005079,0.005158,...,0.001087,0.010811,0.010099,0.012193,2.963117e-10,0.023576,0.0,0.000055,0.000035,0.001155
2020-09-30,0.000765,0.008226,0.004563,1.708035e-17,0.019493,1.346823e-51,0.024273,0.001890,0.003668,0.004464,...,0.001386,0.005405,0.009341,0.011733,1.481558e-10,0.019958,0.0,0.000028,0.000017,0.000578


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-09-28,0.000143,0.018558,0.002913,0.004144,0.051236,0.000005,0.018449,0.001830,0.002096,0.016082,...,0.006225,0.011757,0.009895,0.014763,0.001167,0.036868,1.027163e-33,0.002194,0.003285,0.002797
2020-09-29,0.001665,0.016030,0.002861,0.002072,0.031134,0.000003,0.017150,0.003022,0.002487,0.020727,...,0.006514,0.010437,0.011121,0.014122,0.004588,0.032117,5.135813e-34,0.004652,0.005839,0.002217
2020-09-30,0.001489,0.015451,0.002969,0.067044,0.045027,0.000001,0.016075,0.003244,0.001890,0.020789,...,0.004313,0.010865,0.009298,0.015044,0.003783,0.027267,2.567907e-34,0.006640,0.003709,0.015891


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-28,0.005561,0.015506,NaN,0.001669,0.009220,0.003663,0.007333,0.005263,0.005664,3.389557e-60,...,0.006577,0.004365,0.013820,0.002197,0.002926,0.004384,0.005404,0.011929,0.018171,0.022352
2020-09-29,0.004647,0.015980,NaN,0.002384,0.008912,0.003642,0.007517,0.004224,0.006314,1.694779e-60,...,0.005557,0.006167,0.011767,0.002244,0.001463,0.005268,0.005034,0.011668,0.019151,0.016998
2020-09-30,0.006058,0.014594,NaN,0.001932,0.010148,0.003943,0.007408,0.004039,0.005154,8.473893e-61,...,0.006704,0.006834,0.012163,0.001980,0.000732,0.005169,0.005274,0.010836,0.019241,0.019408


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-09-28      0.006430  0.012139  0.004317  0.003203  0.002526  0.003025   
2020-09-29      0.003775  0.007680  0.002159  0.002363  0.002190  0.001512   
2020-09-30      0.003007  0.006873  0.006723  0.005741  0.003563  0.003227   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-09-28      0.000570  0.005284  0.003802  0.006444  ...  0.016139   
2020-09-29      0.001926  0.003769  0.003181  0.005665  ...  0.012271   
2020-09-30      0.000963  0.003713  0.003293  0.005264  ...  0.016552   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-09-28      0.050185  0.021515  0.023879   0.001877  0.016282  0.005908   
2020-09-29      0.058426  0.015895  0.051940   0.003932  0.008141  0.002954   
2020-09-30      0.044838  0.009642  0.063007   0.006430  0.019249  0.005714   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-09-28      -0.571428  0.002753  0.025009  
2020-09-29      -0.571428  0.005724  0.026393  
2020-09-30      -0.571428  0.002862  0.040224  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-09-28,0.000623,0.028089,NaN,0.001542,0.016226,0.002681,0.001381,0.000272,0.001413,NaN,...,0.005828,0.004263,0.003440,9.816241e-19,0.026316,0.004576,0.000776,0.008875,0.002220,0.000337
2020-09-29,0.003510,0.014044,NaN,0.001482,0.016014,0.006106,0.001180,0.000358,0.001495,NaN,...,0.009402,0.005136,0.006135,4.908121e-19,0.013158,0.004654,0.006102,0.016133,0.007735,0.000169
2020-09-30,0.006324,0.007022,NaN,0.002339,0.015044,0.006316,0.001812,0.000512,0.001535,NaN,...,0.011726,0.006153,0.005256,2.454060e-19,0.006579,0.005467,0.003522,0.015210,0.014252,0.000084


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-09-28      0.000331  0.000675  7.614661e-14  0.000115  0.001154   
2020-09-29      0.040165  0.000337  3.807330e-14  0.000058  0.000577   
2020-09-30      0.020083  0.020169  1.903665e-14  0.050029  0.000289   

Province_State                                                    ...  \
Admin2           Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-09-28      0.000009  0.000414  0.008492  0.000006 -0.009249  ...   
2020-09-29      0.000005  0.013028  0.004246  0.000003 -0.004625  ...   
2020-09-30      0.035717  0.006514  0.002123  0.000002 -0.002312  ...   

Province_State   Wyoming                                                   \
Admin2              Park Platte  Sheridan Sublette Sweetwater Teton Uinta   
2020-09-28      0.000244    0.0  0.000366      0.0        0.0   0.0   0.0   
2020-09-29      0.000122    0.0  0.000183      0.0        0.0   0.0   0.0   
2020-09-30      0.000061    0.0  0.000092      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-09-28      -0.999969      0.0    NaN  
2020-09-29      -0.999969      0.0    NaN  
2020-09-30      -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200930,AK,8780,448427,NaN,457207,53.0,NaN,NaN,NaN,...,457207,0,0,1716aba4b705ff74814829d25487c6e57d27122d,0,0,0,0,0,NaN
1,20200930,AL,154701,994475,NaN,1132039,776.0,17257.0,NaN,1815.0,...,1149176,23,75,6ec6077978a7573b69eb4da4ba7236140c79abd6,0,0,0,0,0,NaN
2,20200930,AR,83697,949107,NaN,1029717,484.0,5354.0,218.0,NaN,...,1032804,19,0,5721c525c20cecd19b5ae38ff1fc45909337796d,0,0,0,0,0,NaN
3,20200930,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,...,1571,0,0,8d45036531d9c5b591541ee8dc6c5108544018b8,0,0,0,0,0,NaN
4,20200930,AZ,218507,1243641,NaN,1457575,560.0,22119.0,115.0,NaN,...,1462148,18,72,fbed080c9c84737d1b3074937b936442064bbaeb,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200930,AK,8780.0,448427.0,NaN,457207.0,53.0,NaN,NaN,NaN,...,457207,0,0,1716aba4b705ff74814829d25487c6e57d27122d,0,0,0,0,0,NaN
1,20200930,AL,154701.0,994475.0,NaN,1132039.0,776.0,17257.0,NaN,1815.0,...,1149176,23,75,6ec6077978a7573b69eb4da4ba7236140c79abd6,0,0,0,0,0,NaN
2,20200930,AR,83697.0,949107.0,NaN,1029717.0,484.0,5354.0,218.0,NaN,...,1032804,19,0,5721c525c20cecd19b5ae38ff1fc45909337796d,0,0,0,0,0,NaN
3,20200930,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,...,1571,0,0,8d45036531d9c5b591541ee8dc6c5108544018b8,0,0,0,0,0,NaN
4,20200930,AZ,218507.0,1243641.0,NaN,1457575.0,560.0,22119.0,115.0,NaN,...,1462148,18,72,fbed080c9c84737d1b3074937b936442064bbaeb,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-30,AK,8780,448427,NaN,457207,53.0,NaN,NaN,NaN,7.0,...,457207,0,0,1716aba4b705ff74814829d25487c6e57d27122d,0,0,0,0,0,NaN
2020-09-30,AL,154701,994475,NaN,1132039,776.0,17257.0,NaN,1815.0,NaN,...,1149176,23,75,6ec6077978a7573b69eb4da4ba7236140c79abd6,0,0,0,0,0,NaN
2020-09-30,AR,83697,949107,NaN,1029717,484.0,5354.0,218.0,NaN,93.0,...,1032804,19,0,5721c525c20cecd19b5ae38ff1fc45909337796d,0,0,0,0,0,NaN
2020-09-30,AS,0,1571,NaN,1571,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,8d45036531d9c5b591541ee8dc6c5108544018b8,0,0,0,0,0,NaN
2020-09-30,AZ,218507,1243641,NaN,1457575,560.0,22119.0,115.0,NaN,55.0,...,1462148,18,72,fbed080c9c84737d1b3074937b936442064bbaeb,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-09-30,AK,8780.0,448427.0,NaN,457207.0,53.0,NaN,NaN,NaN,7.0,...,457207,0,0,1716aba4b705ff74814829d25487c6e57d27122d,0,0,0,0,0,NaN
2020-09-30,AL,154701.0,994475.0,NaN,1132039.0,776.0,17257.0,NaN,1815.0,NaN,...,1149176,23,75,6ec6077978a7573b69eb4da4ba7236140c79abd6,0,0,0,0,0,NaN
2020-09-30,AR,83697.0,949107.0,NaN,1029717.0,484.0,5354.0,218.0,NaN,93.0,...,1032804,19,0,5721c525c20cecd19b5ae38ff1fc45909337796d,0,0,0,0,0,NaN
2020-09-30,AS,0.0,1571.0,NaN,1571.0,NaN,NaN,NaN,NaN,NaN,...,1571,0,0,8d45036531d9c5b591541ee8dc6c5108544018b8,0,0,0,0,0,NaN
2020-09-30,AZ,218507.0,1243641.0,NaN,1457575.0,560.0,22119.0,115.0,NaN,55.0,...,1462148,18,72,fbed080c9c84737d1b3074937b936442064bbaeb,0,0,0,0,0,NaN
